In [10]:
import numpy as np
import pandas as pd
import pandas_profiling
import matplotlib.pyplot as plt

In [24]:
input_folder = "./data"
output_folder = "./output"
profiles_folder = "./profiles"

In [ ]:
#------------ CONFIG ------------#
eda  = False
#--------------------------------#

In [25]:
train = pd.read_csv(f"{input_folder}/train.csv")
test = pd.read_csv(f"{input_folder}/test.csv")

In [30]:
if eda:
    trainprofile = train.profile_report(title='Train Profiling Report', plot={'histogram': {'bins': 8}}, progress_bar=False)
    trainprofile.to_file(output_file=f"{profiles_folder}/train_profiling.html")
    testprofile = test.profile_report(title='Test Profiling Report', plot={'histogram': {'bins': 8}}, progress_bar=False)
    testprofile.to_file(output_file=f"{profiles_folder}/test_profiling.html")

In [ ]:
imputer = KNNImputer()
train = imputer.fit_transform(X)
X = pd.DataFrame(X, columns=all_columns)

In [29]:
#Train data

X = train.drop(["Employee_ID", "Churn_risk"], axis=1)
y = train["Churn_risk"]

In [30]:
#Categorical variables

categorical_columns = ["Department", "Gender"]
categorical_columns.extend(marital_columns)

other_columns = ['Age', 'Days_off', 'Rotations', 'Satis_leader', 'Satis_team', 
                 'Emails', 'Tenure', 'Bonus', 'Distance', 'Kids', 'Overtime']

all_columns = other_columns
all_columns.extend(categorical_columns)

In [31]:
# Missing Values

imputer = KNNImputer()
X = imputer.fit_transform(X)
X = pd.DataFrame(X, columns=all_columns)

test_ids = test["Employee_ID"]
test = pd.DataFrame(imputer.transform(test.drop("Employee_ID", axis=1)), columns=all_columns)
test += test.min().min()

In [32]:
#Scaling

X = pd.DataFrame(X, columns=all_columns)
X += abs(X.min().min())

In [33]:
# Naive Bayes Model

clf1 = MultinomialNB()
clf2 = CategoricalNB()

In [41]:
def kfold(clf1, clf2):
    
    nfolds = 600
    print("# of Folds:", nfolds)
        
    strat = StratifiedShuffleSplit(n_splits=nfolds, test_size=0.25, random_state=1421)
    outcomes = []

    for train_indices, test_indices in strat.split(X, y):
        X_train1, X_test1 = X[other_columns].values[train_indices], X[other_columns].values[test_indices]
        X_train2, X_test2 = X[categorical_columns].values[train_indices], X[categorical_columns].values[test_indices]
        y_train, y_test = y.values[train_indices], y.values[test_indices]

        clf1.fit(X_train1, y_train)
        clf2.fit(X_train2, y_train)        

        predictions1 = clf1.predict_proba(X_test1)
        predictions2 = clf2.predict_proba(X_test2)
        predictions = ((predictions1 + predictions2)/2).argmax(axis=1)

        f1 = f1_score(y_test, predictions, average="micro")
        outcomes.append(f1)

    mean_outcome = np.mean(outcomes)
    return mean_outcome

mean_outcome = kfold(clf1, clf2)
print("mean_outcome:", mean_outcome)

# of Folds: 600
mean_outcome: 0.668775641025641


In [43]:
clf1.fit(X[other_columns], y);
clf2.fit(X[categorical_columns], y);

In [58]:
#predictions

preds1 = clf1.predict_proba(pd.DataFrame(test, columns=all_columns)[other_columns])
preds2 = clf2.predict_proba(pd.DataFrame(test, columns=all_columns)[categorical_columns])
preds = ((preds1 + preds2)/2).argmax(axis=1)
preds = oe_label.inverse_transform(preds.reshape(-1,1)).reshape(-1,)

In [63]:
predictions = pd.DataFrame({"Employee_ID": test_ids, "Churn_risk": preds})
predictions.to_csv('02. Predictions/m20190450_version2.csv', index = False)
predictions.head()

,Employee_ID,Churn_risk
0,1005201,low
1,1005202,medium
2,1005203,medium
3,1005204,medium
4,1005205,medium
